In [2]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [61]:
class Loader:
    def __init__(self):
        pass
    
    def load_dataset(self):    
        
        '''
        citations: [target논문인덱스, source논문인덱스]
        papers: [논문인덱스, 1424개 단어 포함 여부, 주제(subject)]
        train: papers 데이터 중 50% 샘플링
        test: papers 데이터 중 50% 샘플링
        '''
        
        zip_file = keras.utils.get_file(
            fname="cora.tgz",
            origin="https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz",
            extract=True,
        )
        data_dir = os.path.join(os.path.dirname(zip_file), "cora")
        
        citations = pd.read_csv(
            os.path.join(data_dir, "cora.cites"),
            sep="\t",
            header=None,
            names=["target", "source"],
        )
        
        column_names = ["paper_id"] + [f"term_{idx}" for idx in range(1433)] + ["subject"]
        papers = pd.read_csv(
            os.path.join(data_dir, "cora.content"), 
            sep="\t", 
            header=None, 
            names=column_names,
        )
        
        class_values = sorted(papers["subject"].unique())
        class_idx = {name: id for id, name in enumerate(class_values)}
        paper_idx = {name: idx for idx, name in enumerate(sorted(papers["paper_id"].unique()))}

        papers["paper_id"] = papers["paper_id"].apply(lambda name: paper_idx[name])
        citations["source"] = citations["source"].apply(lambda name: paper_idx[name])
        citations["target"] = citations["target"].apply(lambda name: paper_idx[name])
        papers["subject"] = papers["subject"].apply(lambda value: class_idx[value])
        
        train_data, test_data = [], []

        for _, group_data in papers.groupby("subject"):
            # Select around 50% of the dataset for training.
            random_selection = np.random.rand(len(group_data.index)) <= 0.5
            train_data.append(group_data[random_selection])
            test_data.append(group_data[~random_selection])

        train_data = pd.concat(train_data).sample(frac=1)
        test_data = pd.concat(test_data).sample(frac=1)

        print("citations data shape:", citations.shape)
        print("papers data shape:", papers.shape)
        print("Train data shape:", train_data.shape)
        print("Test data shape:", test_data.shape)
        
        feature_names = set(papers.columns) - {"paper_id", "subject"}
        num_features = len(feature_names)
        num_classes = len(class_idx)

        # Create train and test features as a numpy array.
        x_train = train_data[feature_names].to_numpy()
        x_test = test_data[feature_names].to_numpy()
        
        # Create train and test targets as a numpy array.
        y_train = train_data["subject"]
        y_test = test_data["subject"]
        
        return citations, papers, x_train, x_test, y_train, y_test, train_data, test_data
    
    

In [62]:
citations, papers, x_train, x_test, y_train, y_test, train_data, test_data = Loader().load_dataset()

citations data shape: (5429, 2)
papers data shape: (2708, 1435)
Train data shape: (1317, 1435)
Test data shape: (1391, 1435)


In [63]:
# Create an edges array (sparse adjacency matrix) of shape [2, num_edges].
edges = citations[["source", "target"]].to_numpy().T
# Create an edge weights array of ones.
edge_weights = tf.ones(shape=edges.shape[1])
# Create a node features array of shape [num_nodes, num_features].
node_features = tf.cast(
    papers.sort_values("paper_id")[feature_names].to_numpy(), dtype=tf.dtypes.float32
)
# Create graph info tuple with node_features, edges, and edge_weights.
graph_info = (node_features, edges, edge_weights)

print("Edges shape:", edges.shape)
print("Nodes shape:", node_features.shape)

Edges shape: (2, 5429)
Nodes shape: (2708, 1433)


In [65]:
def create_ffn(hidden_units, dropout_rate, name=None):
    '''
hidden_units = [32, 32]
dropout_rate = 0.2
'''

    fnn_layers = []

    for units in hidden_units:
        fnn_layers.append(layers.BatchNormalization())
        fnn_layers.append(layers.Dropout(dropout_rate))
        fnn_layers.append(layers.Dense(units, activation=tf.nn.gelu))

    return keras.Sequential(fnn_layers, name=name)

In [66]:
class GraphConvLayer(layers.Layer):
    def __init__(
        self,
        hidden_units,
        dropout_rate=0.2,
        aggregation_type="mean",
        combination_type="concat",
        normalize=False,
        *args,
        **kwargs,
    ):
        super(GraphConvLayer, self).__init__(*args, **kwargs)

        self.aggregation_type = aggregation_type
        self.combination_type = combination_type
        self.normalize = normalize

        self.ffn_prepare = create_ffn(hidden_units, dropout_rate)
        if self.combination_type == "gated":
            self.update_fn = layers.GRU(
                units=hidden_units,
                activation="tanh",
                recurrent_activation="sigmoid",
                dropout=dropout_rate,
                return_state=True,
                recurrent_dropout=dropout_rate,
            )
        else:
            self.update_fn = create_ffn(hidden_units, dropout_rate)

    def prepare(self, node_repesentations, weights=None):
        # node_repesentations shape is [num_edges, embedding_dim].
        messages = self.ffn_prepare(node_repesentations)
        if weights is not None:
            messages = messages * tf.expand_dims(weights, -1)
        return messages

    def aggregate(self, node_indices, neighbour_messages):
        # node_indices shape is [num_edges].
        # neighbour_messages shape: [num_edges, representation_dim].
        num_nodes = tf.math.reduce_max(node_indices) + 1
        if self.aggregation_type == "sum":
            aggregated_message = tf.math.unsorted_segment_sum(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        elif self.aggregation_type == "mean":
            aggregated_message = tf.math.unsorted_segment_mean(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        elif self.aggregation_type == "max":
            aggregated_message = tf.math.unsorted_segment_max(
                neighbour_messages, node_indices, num_segments=num_nodes
            )
        else:
            raise ValueError(f"Invalid aggregation type: {self.aggregation_type}.")

        return aggregated_message

    def update(self, node_repesentations, aggregated_messages):
        # node_repesentations shape is [num_nodes, representation_dim].
        # aggregated_messages shape is [num_nodes, representation_dim].
        if self.combination_type == "gru":
            # Create a sequence of two elements for the GRU layer.
            h = tf.stack([node_repesentations, aggregated_messages], axis=1)
        elif self.combination_type == "concat":
            # Concatenate the node_repesentations and aggregated_messages.
            h = tf.concat([node_repesentations, aggregated_messages], axis=1)
        elif self.combination_type == "add":
            # Add node_repesentations and aggregated_messages.
            h = node_repesentations + aggregated_messages
        else:
            raise ValueError(f"Invalid combination type: {self.combination_type}.")

        # Apply the processing function.
        node_embeddings = self.update_fn(h)
        if self.combination_type == "gru":
            node_embeddings = tf.unstack(node_embeddings, axis=1)[-1]

        if self.normalize:
            node_embeddings = tf.nn.l2_normalize(node_embeddings, axis=-1)
        return node_embeddings

    def call(self, inputs):
        """Process the inputs to produce the node_embeddings.

        inputs: a tuple of three elements: node_repesentations, edges, edge_weights.
        Returns: node_embeddings of shape [num_nodes, representation_dim].
        """

        node_repesentations, edges, edge_weights = inputs
        # Get node_indices (source) and neighbour_indices (target) from edges.
        node_indices, neighbour_indices = edges[0], edges[1]
        # neighbour_repesentations shape is [num_edges, representation_dim].
        neighbour_repesentations = tf.gather(node_repesentations, neighbour_indices)

        # Prepare the messages of the neighbours.
        neighbour_messages = self.prepare(neighbour_repesentations, edge_weights)
        # Aggregate the neighbour messages.
        aggregated_messages = self.aggregate(node_indices, neighbour_messages)
        # Update the node embedding with the neighbour messages.
        return self.update(node_repesentations, aggregated_messages)

In [67]:
class GNNNodeClassifier(tf.keras.Model):
    def __init__(
        self,
        graph_info,
        num_classes,
        hidden_units,
        aggregation_type="sum",
        combination_type="concat",
        dropout_rate=0.2,
        normalize=True,
        *args,
        **kwargs,
    ):
        super(GNNNodeClassifier, self).__init__(*args, **kwargs)

        # Unpack graph_info to three elements: node_features, edges, and edge_weight.
        node_features, edges, edge_weights = graph_info
        self.node_features = node_features
        self.edges = edges
        self.edge_weights = edge_weights
        # Set edge_weights to ones if not provided.
        if self.edge_weights is None:
            self.edge_weights = tf.ones(shape=edges.shape[1])
        # Scale edge_weights to sum to 1.
        self.edge_weights = self.edge_weights / tf.math.reduce_sum(self.edge_weights)

        # Create a process layer.
        self.preprocess = create_ffn(hidden_units, dropout_rate, name="preprocess")
        # Create the first GraphConv layer.
        self.conv1 = GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv1",
        )
        # Create the second GraphConv layer.
        self.conv2 = GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv2",
        )
        # Create a postprocess layer.
        self.postprocess = create_ffn(hidden_units, dropout_rate, name="postprocess")
        # Create a compute logits layer.
        self.compute_logits = layers.Dense(units=num_classes, name="logits")

    def call(self, input_node_indices):
        # Preprocess the node_features to produce node representations.
        x = self.preprocess(self.node_features)
        # Apply the first graph conv layer.
        x1 = self.conv1((x, self.edges, self.edge_weights))
        # Skip connection.
        x = x1 + x
        # Apply the second graph conv layer.
        x2 = self.conv2((x, self.edges, self.edge_weights))
        # Skip connection.
        x = x2 + x
        # Postprocess node embedding.
        x = self.postprocess(x)
        # Fetch node embeddings for the input node_indices.
        node_embeddings = tf.gather(x, input_node_indices)
        # Compute logits
        return self.compute_logits(node_embeddings)

In [68]:
hidden_units = [32, 32]
learning_rate = 0.01
dropout_rate = 0.5
num_epochs = 300
batch_size = 256

gnn_model = GNNNodeClassifier(
    graph_info=graph_info,
    num_classes=num_classes,
    hidden_units=hidden_units,
    dropout_rate=dropout_rate,
    name="gnn_model",
)

print("GNN output shape:", gnn_model([1, 10, 100]))

gnn_model.summary()

GNN output shape: tf.Tensor(
[[ 0.04023143 -0.03798582 -0.01061788  0.03870849  0.08173378  0.06705542
   0.0176754 ]
 [-0.0204511  -0.00529465  0.03362372 -0.05983638  0.19713391 -0.0527336
  -0.10019694]
 [ 0.0660347  -0.0315948   0.01978562  0.03459518  0.05062745  0.08475165
   0.02827925]], shape=(3, 7), dtype=float32)
Model: "gnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 preprocess (Sequential)     (2708, 32)                52804     
                                                                 
 graph_conv1 (GraphConvLayer  multiple                 5888      
 )                                                               
                                                                 
 graph_conv2 (GraphConvLayer  multiple                 5888      
 )                                                               
                                                             

In [69]:


def run_experiment(model, x_train, y_train):
    # Compile the model.
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    )
    # Create an early stopping callback.
    early_stopping = keras.callbacks.EarlyStopping(
        monitor="val_acc", patience=50, restore_best_weights=True
    )
    # Fit the model.
    history = model.fit(
        x=x_train,
        y=y_train,
        epochs=num_epochs,
        batch_size=batch_size,
        validation_split=0.15,
        callbacks=[early_stopping],
    )

    return history

In [70]:
x_train = train_data.paper_id.to_numpy()
history = run_experiment(gnn_model, x_train, y_train)

Epoch 1/300
5/5 [==============================] - 3s 162ms/step - loss: 2.2650 - acc: 0.1537 - val_loss: 1.9017 - val_acc: 0.3131
Epoch 2/300
5/5 [==============================] - 0s 69ms/step - loss: 1.9851 - acc: 0.2377 - val_loss: 1.8790 - val_acc: 0.3131
Epoch 3/300
5/5 [==============================] - 0s 68ms/step - loss: 1.9357 - acc: 0.2681 - val_loss: 1.8767 - val_acc: 0.3131
Epoch 4/300
5/5 [==============================] - 0s 75ms/step - loss: 1.8984 - acc: 0.2645 - val_loss: 1.8758 - val_acc: 0.3131
Epoch 5/300
5/5 [==============================] - 0s 72ms/step - loss: 1.8701 - acc: 0.2672 - val_loss: 1.8694 - val_acc: 0.3131
Epoch 6/300
5/5 [==============================] - 0s 71ms/step - loss: 1.8651 - acc: 0.2744 - val_loss: 1.8586 - val_acc: 0.3131
Epoch 7/300
5/5 [==============================] - 0s 74ms/step - loss: 1.8614 - acc: 0.2547 - val_loss: 1.8499 - val_acc: 0.3131
Epoch 8/300
5/5 [==============================] - 0s 70ms/step - loss: 1.8309 - acc: 0.2